In [467]:
using LowLevelFEM, LinearAlgebra, SparseArrays

In [468]:
function poissonMatrixVector(
    problem::Problem;
    coefficient::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # elementwise tensor field (node values gathered per element)
    S = nodesToElements(coefficient)
    Se = Dict(zip(S.numElem, S.A))

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- get Lagrange shape functions for proper interpolation
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- gradients
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # tensor at element nodes (9×numNodes)
                    Snode = Se[elem]

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate S to Gauss point using shape functions
                        Sgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Sa = reshape(Snode[9a-8:9a, 1], dim, dim)
                            Sgp .+= H[a, k] * Sa
                        end

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += ∇Na[i] * Sgp[i, j] * ∇Nb[j] * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


poissonMatrixVector (generic function with 1 method)

In [469]:
function gradDivMatrixF(
    problem::Problem;
    coefficient::Union{Number,ScalarField},
    F::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # --- elementwise nodal tensors for F
    Fe = nodesToElements(F)
    Fe_map = Dict(zip(Fe.numElem, Fe.A))

    # --- elementwise nodal scalars for λ (if needed)
    λ_is_scalarfield = !(coefficient isa Number)
    if λ_is_scalarfield
        λe = nodesToElements(coefficient)
        λmap = Dict(zip(λe.numElem, λe.A))   # each entry: (numNodes×1) vector per element
    else
        λconst = Float64(coefficient)
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # nodal data for this element
                    Felem = Fe_map[elem]
                    if λ_is_scalarfield
                        λa = λmap[elem][:, 1]  # length = numNodes
                    end

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate F to GP using shape functions
                        Fgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Fgp .+= Na * reshape(Felem[9a-8:9a, 1], dim, dim)
                        end

                        # --- interpolate λ to GP if needed
                        λgp = λ_is_scalarfield ? dot(λa, H[:, k]) : λconst

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += λgp * (Fgp[i, i] * ∇Na[i]) * (Fgp[j, j] * ∇Nb[j]) * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


gradDivMatrixF (generic function with 1 method)

In [470]:
function poissonMatrixSymGradF(
    problem::Problem;
    coefficient::Union{Number,ScalarField},
    F::TensorField)

    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # --- elementwise nodal tensors for F
    Fe = nodesToElements(F)
    Fe_map = Dict(zip(Fe.numElem, Fe.A))

    # --- elementwise nodal scalars for μ (if needed)
    μ_is_scalarfield = !(coefficient isa Number)
    if μ_is_scalarfield
        μe = nodesToElements(coefficient)
        μmap = Dict(zip(μe.numElem, μe.A))  # each entry: (numNodes×1) vector per element
    else
        μconst = Float64(coefficient)
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                intPoints, intWeights =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation of F and μ)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, intPoints, "GradLagrange")
                ∇h = reshape(dfun, :, length(intWeights))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, intPoints)
                    Jac = reshape(jac, 3, :)

                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    # nodal data for this element
                    Felem = Fe_map[elem]
                    if μ_is_scalarfield
                        μa = μmap[elem][:, 1]   # length = numNodes
                    end

                    for k in eachindex(intWeights)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * intWeights[k]

                        # --- interpolate F to Gauss point using shape functions
                        Fgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Fgp .+= Na * reshape(Felem[9a-8:9a, 1], dim, dim)
                        end

                        # --- interpolate μ to Gauss point if needed
                        μgp = μ_is_scalarfield ? dot(μa, H[:, k]) : μconst

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            GradNa = zeros(dim, dim)
                            GradNb = zeros(dim, dim)
                            for i in 1:dim
                                GradNa[i, i] = ∇Na[i]
                                GradNb[i, i] = ∇Nb[i]
                            end

                            Ea = 0.5 * (Fgp' * GradNa + GradNa' * Fgp)
                            Eb = 0.5 * (Fgp' * GradNb + GradNb' * Fgp)

                            for i in 1:dim, j in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + j
                                Ke[ia, ib] += 2 * μgp * Ea[i, j] * Eb[i, j] * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


poissonMatrixSymGradF (generic function with 1 method)

In [471]:
function materialTangentTL_FC(
    problem::Problem;
    F::TensorField,
    C::AbstractMatrix
)
    @assert problem.dim == 3
    @assert problem.pdim == 3
    @assert size(C) == (6,6)
    @assert all(x -> (x isa Number) || (x isa ScalarField), C)

    gmsh.model.setCurrent(problem.name)

    dim  = 3
    pdim = 3
    dof  = pdim * problem.non

    # --- elementwise nodal F (9×numNodes per element)
    Fe   = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    # --- Preprocess C entries:
    # For ScalarField entries: build element->nodal-values map once.
    Centry = Matrix{Any}(undef, 6, 6)
    for I in CartesianIndices(C)
        cij = C[I]
        if cij isa ScalarField
            ce = nodesToElements(cij)
            Centry[I] = Dict(zip(ce.numElem, ce.A))  # elem => (numNodes×1) nodal values
        else
            Centry[I] = Float64(cij)
        end
    end

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Int[]
    J = Int[]
    V = Float64[]
    sizehint!(I, lengthOfIJV)
    sizehint!(J, lengthOfIJV)
    sizehint!(V, lengthOfIJV)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # shape functions for interpolation
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # gradients
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Fnode = Fmap[elem]
                    Ke = zeros(pdim*numNodes, pdim*numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:3, 3k-2:3k])'
                        w    = jacDet[k] * wip[k]

                        # --- F at Gauss: Fgp = Σ_a N_a F_a
                        Fgp = zeros(3,3)
                        for a in 1:numNodes
                            Na = H[a,k]
                            Fgp .+= Na * reshape(Fnode[9a-8:9a, 1], 3, 3)
                        end

                        # --- C at Gauss: Cgp_ij = Σ_a N_a Cij_a  (ha ScalarField)
                        Cgp = zeros(6,6)
                        for ii in 1:6, jj in 1:6
                            cij = Centry[ii,jj]
                            if cij isa Float64
                                Cgp[ii,jj] = cij
                            else
                                # dict elem => nodalValues(numNodes×1)
                                nod = cij[elem][:,1]
                                Cgp[ii,jj] = dot(nod, H[:,k])
                            end
                        end

                        # --- build B(F)  (6 × 3numNodes)
                        # Column corresponds to dof (a,j): δu_{a,j}
                        # δF = e_j ⊗ ∇N_a^T
                        # δE = sym(F^T δF) = 1/2 (F^T δF + δF^T F)
                        B = zeros(6, 3*numNodes)

                        for a in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a, k]  # 3-vector

                            for j in 1:3
                                # v = F^T e_j = row j of F, as a column vector
                                v = @view Fgp[j, :]  # length-3 row view

                                # dE = 1/2 (v ⊗ ∇Na^T + ∇Na ⊗ v^T)
                                dE11 = 0.5*(v[1]*∇Na[1] + ∇Na[1]*v[1])
                                dE22 = 0.5*(v[2]*∇Na[2] + ∇Na[2]*v[2])
                                dE33 = 0.5*(v[3]*∇Na[3] + ∇Na[3]*v[3])

                                dE23 = 0.5*(v[2]*∇Na[3] + ∇Na[2]*v[3])
                                dE13 = 0.5*(v[1]*∇Na[3] + ∇Na[1]*v[3])
                                dE12 = 0.5*(v[1]*∇Na[2] + ∇Na[1]*v[2])

                                col = (a-1)*3 + j
                                B[1, col] = dE11
                                B[2, col] = dE22
                                B[3, col] = dE33
                                B[4, col] = 2*dE23
                                B[5, col] = 2*dE13
                                B[6, col] = 2*dE12
                            end
                        end

                        Ke .+= (B' * Cgp * B) * w
                    end

                    # scatter Ke
                    for a in 1:(3*numNodes), b in 1:(3*numNodes)
                        Ig = (nodeTags[div(a-1,3)+1]-1)*3 + mod(a-1,3)+1
                        Jg = (nodeTags[div(b-1,3)+1]-1)*3 + mod(b-1,3)+1
                        push!(I, Ig); push!(J, Jg); push!(V, Ke[a,b])
                    end
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


materialTangentTL_FC (generic function with 1 method)

In [472]:
function initialStressMatrixTL(
    problem::Problem;
    stress::TensorField   # P, S, σ – a kód nem tudja, nem is érdekli
)
    @assert problem.pdim == problem.dim
    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    # elementwise nodal stresses
    Se = nodesToElements(stress)
    Smap = Dict(zip(Se.numElem, Se.A))

    lengthOfIJV = LowLevelFEM.estimateLengthOfIJV(problem)
    I = Vector{Int}(undef, lengthOfIJV)
    J = Vector{Int}(undef, lengthOfIJV)
    V = Vector{Float64}(undef, lengthOfIJV)
    pos = 1

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for stress interpolation)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Selem = Smap[elem]
                    Ke = zeros(pdim * numNodes, pdim * numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- interpolate stress to Gauss point (CORRECT)
                        Sgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Sgp .+= Na * reshape(Selem[9a-8:9a, 1], dim, dim)
                        end

                        for a in 1:numNodes, b in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                            # scalar geometric stiffness contribution
                            g = dot(∇Na, Sgp * ∇Nb)

                            # distribute to displacement components (i = i only)
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                ib = (b - 1) * pdim + i
                                Ke[ia, ib] += g * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                        I[pos] = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                        J[pos] = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                        V[pos] = Ke[a, b]
                        pos += 1
                    end
                end
            end
        end
    end

    resize!(I, pos - 1)
    resize!(J, pos - 1)
    resize!(V, pos - 1)

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


initialStressMatrixTL (generic function with 1 method)

In [473]:
function internalForceTL(problem::Problem, P::TensorField)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    Pe = nodesToElements(P)
    Pmap = Dict(zip(Pe.numElem, Pe.A))

    f = zeros(dof)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags =
                gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, order, numNodes, _, _ =
                    gmsh.model.mesh.getElementProperties(et)

                ip, wip =
                    gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

                # --- Lagrange shape functions (for proper interpolation of P)
                _, hfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
                H = reshape(hfun, numNodes, :)

                # --- GradLagrange
                _, dfun, _ =
                    gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
                ∇h = reshape(dfun, :, length(wip))

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    jac, jacDet, _ =
                        gmsh.model.mesh.getJacobian(elem, ip)
                    Jac = reshape(jac, 3, :)

                    Pelem = Pmap[elem]
                    fe = zeros(pdim * numNodes)

                    for k in eachindex(wip)
                        invJ = inv(Jac[1:dim, 3k-2:3k])'
                        w = jacDet[k] * wip[k]

                        # --- interpolate P to Gauss point using shape functions
                        Pgp = zeros(dim, dim)
                        for a in 1:numNodes
                            Na = H[a, k]
                            Pgp .+= Na * reshape(Pelem[9a-8:9a, 1], dim, dim)
                        end

                        for a in 1:numNodes
                            ∇Na = invJ * ∇h[3a-2:3a-(3-dim), k]
                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                fe[ia] += dot(Pgp[i, :], ∇Na) * w
                            end
                        end
                    end

                    # scatter
                    for a in 1:numNodes, i in 1:dim
                        Ig = (nodeTags[a] - 1) * pdim + i
                        f[Ig] += fe[(a-1)*pdim+i]
                    end
                end
            end
        end
    end

    if pdim ≠ 3
        error("dim ≠ 3 is not implemented yet")
    end
    return VectorField([], reshape(f, :, 1), [0.0], [], 1, :v3D, problem)
end


internalForceTL (generic function with 1 method)

In [474]:
function externalTangentFollowerTL(
    problem::Problem;
    F::TensorField,
    traction_phName::AbstractString,
    t_spatial)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    non = problem.non
    dof = pdim * non

    Fe = nodesToElements(F)
    Fmap = Dict(zip(Fe.numElem, Fe.A))

    Id = zeros(dim, dim)
    for i in 1:dim
        Id[i, i] = 1
    end

    I = Int[]
    J = Int[]
    V = Float64[]

    for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(traction_phName)
        elemTypes, elemTags, elemNodeTags =
            gmsh.model.mesh.getElements(edim, etag)

        for it in eachindex(elemTypes)
            et = elemTypes[it]
            _, _, order, numNodes, _, _ =
                gmsh.model.mesh.getElementProperties(et)

            ip, wip =
                gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

            # Lagrange shape functions
            _, hfun, _ =
                gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
            H = reshape(hfun, numNodes, :)

            # GradLagrange
            _, dfun, _ =
                gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
            ∇h = reshape(dfun, :, length(wip))

            for (e, elem) in enumerate(elemTags[it])
                nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                jac, jacDet, _ =
                    gmsh.model.mesh.getJacobian(elem, ip)
                Jac = reshape(jac, 3, :)

                Fnode = Fmap[elem]
                Ke = zeros(pdim * numNodes, pdim * numNodes)

                for k in eachindex(wip)
                    invJ = inv(Jac[1:dim, 3k-2:3k])'
                    w = jacDet[k] * wip[k]

                    # --- interpolate F to Gauss point using shape functions
                    Fgp = zeros(dim, dim)
                    for a in 1:numNodes
                        Na = H[a, k]
                        Fgp .+= Na * reshape(Fnode[9a-8:9a, 1], dim, dim)
                    end

                    Jgp = det(Fgp)
                    Finv = inv(Fgp)
                    FinvT = Finv'

                    if t_spatial isa Function
                        tgp = t_spatial(0.0, 0.0, 0.0)
                    else
                        tgp = collect(Float64, t_spatial)
                    end

                    for a in 1:numNodes, b in 1:numNodes
                        ∇Nb = invJ * ∇h[3b-2:3b-(3-dim), k]

                        # δF from δu_bj
                        for j in 1:dim
                            A = reshape(Finv[:, j], dim, 1) *
                                reshape(∇Nb, 1, dim)

                            dJFmT =
                                Jgp * FinvT * (tr(A) * Id - A')

                            dt0 = dJFmT * tgp

                            for i in 1:dim
                                ia = (a - 1) * pdim + i
                                jb = (b - 1) * pdim + j
                                Ke[ia, jb] += H[a, k] * dt0[i] * w
                            end
                        end
                    end
                end

                # scatter Ke
                for a in 1:pdim*numNodes, b in 1:pdim*numNodes
                    Ig = (nodeTags[div(a - 1, pdim)+1] - 1) * pdim + mod(a - 1, pdim) + 1
                    Jg = (nodeTags[div(b - 1, pdim)+1] - 1) * pdim + mod(b - 1, pdim) + 1
                    push!(I, Ig)
                    push!(J, Jg)
                    push!(V, Ke[a, b])
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)
    return SystemMatrix(K, problem)
end


externalTangentFollowerTL (generic function with 1 method)

In [475]:
function elementInternalForceTL(problem::Problem, et::Int, elem::Int, nodeTags::Vector{Int};
    P_of_F)

    dim = problem.dim
    pdim = problem.pdim
    @assert dim == 3 && pdim == 3

    _, _, order, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)
    ip, wip = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

    # shape & grad
    _, hfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
    H = reshape(hfun, numNodes, :)
    _, dfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
    ∇h = reshape(dfun, :, length(wip))

    # coords of element nodes (reference)
    # gmsh returns node coords globally; itt feltételezem van segédfüggvényed:
    # X = getNodeCoords(problem, nodeTags) :: (3×numNodes)
    X = nodeCoordinates(problem, nodeTags)  # <-- ha nálad más a neve, írd át

    fe = zeros(pdim * numNodes)

    jac, jacDet, _ = gmsh.model.mesh.getJacobian(elem, ip)
    Jac = reshape(jac, 3, :)

    for k in eachindex(wip)
        invJ = inv(Jac[1:dim, 3k-2:3k])'
        w = jacDet[k] * wip[k]

        # GradN in reference
        GradN = zeros(3, numNodes)
        for a in 1:numNodes
            GradN[:, a] = invJ * ∇h[3a-2:3a, k]
        end

        # deformation gradient from current u (u is encoded in current nodal positions x = X + u)
        # Here we assume caller already provides current nodal positions x (or u_e).
        # We'll compute F = x ⊗ GradN + I  form:
        # F = sum_a x_a ⊗ GradN_a   (with x_a current position vector)
        # Caller will provide current x in a closure; see next function.

        # We'll leave F computation to closure by passing (X, GradN, k) if you want.
        # For simplicity: P_of_F is expected to be called in the Ke-builder where x is known.

        # This function will be used from Ke-builder where we know x.
        # So here we just return fe placeholder.
    end

    return fe
end


elementInternalForceTL (generic function with 1 method)

In [476]:
function elementMaterialTangentFD_TL(problem::Problem, et::Int, elem::Int, nodeTags::Vector{Int};
    P_of_F,
    h=1e-8)

    dim = problem.dim
    pdim = problem.pdim
    @assert dim == 3 && pdim == 3

    _, _, order, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)
    ip, wip = gmsh.model.mesh.getIntegrationPoints(et, "Gauss" * string(2order + 1))

    _, hfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "Lagrange")
    H = reshape(hfun, numNodes, :)
    _, dfun, _ = gmsh.model.mesh.getBasisFunctions(et, ip, "GradLagrange")
    ∇h = reshape(dfun, :, length(wip))

    X = nodeCoordinates(problem, nodeTags)  # 3×numNodes reference coords

    jac, jacDet, _ = gmsh.model.mesh.getJacobian(elem, ip)
    Jac = reshape(jac, 3, :)

    # fe(u_e) black-box
    function fe_of_ue(ue::Vector{Float64})
        # current positions x = X + u
        x = copy(X)
        for a in 1:numNodes
            x[:, a] .+= ue[(a-1)*3+1:(a-1)*3+3]
        end

        fe = zeros(3 * numNodes)

        for k in eachindex(wip)
            invJ = inv(Jac[1:dim, 3k-2:3k])'
            w = jacDet[k] * wip[k]

            # GradN in reference
            GradN = zeros(3, numNodes)
            for a in 1:numNodes
                GradN[:, a] = invJ * ∇h[3a-2:3a, k]
            end

            # F = sum_a x_a ⊗ GradN_a
            F = zeros(3, 3)
            for a in 1:numNodes
                F .+= x[:, a] * (GradN[:, a])'
            end

            P = P_of_F(F)  # 3×3

            # fe_a += (P^T * GradN_a) * w   (equiv to P : ∇N)
            # Your internalForceTL used: fe[ia] += dot(P[i,:], ∇Na) * w
            for a in 1:numNodes
                t = P * GradN[:, a]  # vector length 3
                ia = (a - 1) * 3
                fe[ia+1] += t[1] * w
                fe[ia+2] += t[2] * w
                fe[ia+3] += t[3] * w
            end
        end

        return fe
    end

    ndofe = 3 * numNodes
    Ke = zeros(ndofe, ndofe)

    ue0 = zeros(ndofe)  # caller will replace with actual element displacement when assembling
    f0 = fe_of_ue(ue0)

    for j in 1:ndofe
        ue = copy(ue0)
        ue[j] += h
        fj = fe_of_ue(ue)
        Ke[:, j] = (fj - f0) / h
    end

    return Ke, f0
end


elementMaterialTangentFD_TL (generic function with 1 method)

In [477]:
function materialTangentFD_TL(problem::Problem, u::VectorField; P_of_F, h=1e-8)

    gmsh.model.setCurrent(problem.name)

    dim = problem.dim
    pdim = problem.pdim
    dof = pdim * problem.non

    I = Int[]
    J = Int[]
    V = Float64[]
    f = zeros(dof)

    for mat in problem.material
        for (edim, etag) in gmsh.model.getEntitiesForPhysicalName(mat.phName)

            elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(edim, etag)

            for it in eachindex(elemTypes)
                et = elemTypes[it]
                _, _, _, numNodes, _, _ = gmsh.model.mesh.getElementProperties(et)

                for (e, elem) in enumerate(elemTags[it])
                    nodeTags = elemNodeTags[it][(e-1)*numNodes+1:e*numNodes]

                    # extract element displacement ue from global u
                    ue = zeros(3 * numNodes)
                    for a in 1:numNodes
                        g = nodeTags[a]
                        ue[(a-1)*3+1:(a-1)*3+3] .= u.a[(g-1)*3+1:(g-1)*3+3, 1]
                    end

                    # build Ke and fe at this ue
                    Ke, fe0 = elementMaterialTangentFD_TL(problem, Int(et), Int(elem), Vector{Int}(nodeTags); P_of_F=P_of_F, h=h)

                    # scatter fe
                    for a in 1:numNodes
                        g = nodeTags[a]
                        f[(g-1)*3+1:(g-1)*3+3] .+= fe0[(a-1)*3+1:(a-1)*3+3]
                    end

                    # scatter Ke
                    for a in 1:3*numNodes, b in 1:3*numNodes
                        Ig = (nodeTags[div(a - 1, 3)+1] - 1) * 3 + mod(a - 1, 3) + 1
                        Jg = (nodeTags[div(b - 1, 3)+1] - 1) * 3 + mod(b - 1, 3) + 1
                        push!(I, Ig)
                        push!(J, Jg)
                        push!(V, Ke[a, b])
                    end
                end
            end
        end
    end

    K = sparse(I, J, V, dof, dof)
    dropzeros!(K)

    return SystemMatrix(K, problem), VectorField([], reshape(f, :, 1), [0.0], [], 1, :v3D, problem)
end


materialTangentFD_TL (generic function with 1 method)

In [478]:
gmsh.initialize()
gmsh.open("cube1.geo")

mat = Material("cube", E=2e1)
prob = Problem([mat], type=:VectorField)

Info    : Reading 'cube1.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00157558s, CPU 0.001562s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000270733s, CPU 0.00027s)
Info    : Meshing 3D..

Problem("cube1", :VectorField, 3, 3, Material[Material("cube", :Hooke, 20.0, 0.3, 11.538461538461537, 7.692307692307692, 16.666666666666664, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 6859, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [479]:
F = TensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1]);

In [480]:
#Fleft = nodesToElements(elementsToNodes(F), onPhysicalGroup="left")

#Kext = externalTangentFollowerTL(prob; F=Fleft, traction_phName="left", t_spatial=[1, 0, 0])

In [481]:
function check_fint_Kint_consistency(
    problem,
    u::VectorField;
    bc,
    C,
    ε=1e-7,
    verbose=true
)

    # --- aktuális állapot ---
    F = u ∘ ∇
    I = unitTensor(F)
    E = (F' * F - I) / 2
    S = λ * trace(E) * I + 2μ * E
    P = F * S

    f0 = internalForceTL(problem, P).a[:, 1]

    # --- tangens ---
    #Kmat =
    #    poissonMatrixSymGradF(problem; coefficient=μ, F=F).A +
    #    gradDivMatrixF(problem; coefficient=λ, F=F).A

    #Kgeo = initialStressMatrixTL(problem; stress=P).A
    Kint = materialTangentTL_FC(problem, F=F, C=C)

    # --- perturbáció csak szabad DoF-okon ---
    fixed = constrainedDoFs(problem, bc)
    free = setdiff(1:length(f0), fixed)

    δu = zeros(length(f0))
    δu[free] .= randn(length(free))
    δu ./= norm(δu)

    # --- numerikus derivált ---
    u_pert = copy(u)
    u_pert.a[:, 1] .+= ε .* δu

    #Fp = deformationGradient(problem, u_pert)
    Fp = u_pert ∘ ∇
    Ep = (Fp' * Fp - I) / 2
    Sp = λ * trace(Ep) * I + 2μ * Ep
    Pp = Fp * Sp

    f1 = internalForceTL(problem, Pp).a[:, 1]

    df_fd = (f1 - f0) / ε
    df_lin = Kint.A * δu

    rel_err = norm(df_fd - df_lin) / max(norm(df_fd), 1e-14)

    if verbose
        println("‖df_fd‖  = ", norm(df_fd))
        println("‖Kδu‖    = ", norm(df_lin))
        println("rel err = ", rel_err)
    end

    return rel_err
end


check_fint_Kint_consistency (generic function with 1 method)

In [482]:
suppX = BoundaryCondition("left", ux=0, uy=0, uz=0)
suppY = BoundaryCondition("bottom", uy=0)
suppZ = BoundaryCondition("rear", uz=0)

load1 = BoundaryCondition("frontright", fy=-1)
load2 = BoundaryCondition("rearright", fy=1)

BoundaryCondition("rearright", nothing, nothing, nothing, nothing, nothing, nothing, nothing, 1, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [483]:
μ = mat.μ
λ = mat.λ

11.538461538461537

In [484]:
f_ext = loadVector(prob, [load1]) + loadVector(prob, [load2])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [485]:
showDoFResults(f_ext, name="f")

36

In [486]:
u = vectorField(prob, "cube", [0, 0, 0])

nodal VectorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [487]:
F = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [488]:
I = tensorField(prob, "cube", [1 0 0; 0 1 0; 0 0 1])

nodal TensorField
[1.0; 0.0; … ; 0.0; 1.0;;]

In [489]:
P = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])
S = tensorField(prob, "cube", [0 0 0; 0 0 0; 0 0 0])

nodal TensorField
[0.0; 0.0; … ; 0.0; 0.0;;]

In [490]:
λ0 = λ
μ0 = μ
C1 = [λ0+2μ0 λ0 λ0 0 0 0;
    λ0 λ0+2μ0 λ0 0 0 0;
    λ0 λ0 λ0+2μ0 0 0 0;
    0 0 0 μ0 0 0;
    0 0 0 0 μ0 0;
    0 0 0 0 0 μ0]

6×6 Matrix{Float64}:
 26.9231  11.5385  11.5385  0.0      0.0      0.0
 11.5385  26.9231  11.5385  0.0      0.0      0.0
 11.5385  11.5385  26.9231  0.0      0.0      0.0
  0.0      0.0      0.0     7.69231  0.0      0.0
  0.0      0.0      0.0     0.0      7.69231  0.0
  0.0      0.0      0.0     0.0      0.0      7.69231

## Innen indul az iteráció

In [584]:
#Kμ = poissonMatrixSymGradF(prob, coefficient=μ, F=F)
#Kλ = gradDivMatrixF(prob, coefficient=λ, F=F)

#Kmat = Kμ + Kλ
Kmat = materialTangentTL_FC(prob, F=F, C=C1)

Kgeo = initialStressMatrixTL(prob, stress=S)

Kint = Kmat + Kgeo

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [0.7274743931676745, 0.022308003086846287, -0.12258809254420222, -0.04100756913019919, -0.021779778668838474, -9.48751253957685e-5, -0.2172638340294011, 0.06056129974537203, 0.01069768464187514, -0.0418227088196642  …  -2.561977993482607, -0.23459281669054008, -2.3896843131915406, -20.992332474366066, -0.05021990469177563, 0.041614503779367526, 6.819130650226614, 0.0005479816612616005, 0.06440459756337212, 58.65257550475568], 20577, 20577)

In [585]:
Ffrontright = nodesToElements(elementsToNodes(F), onPhysicalGroup="frontright")
Frearright = nodesToElements(elementsToNodes(F), onPhysicalGroup="rearright")

Kext1 = externalTangentFollowerTL(
    prob;
    F=Ffrontright,
    traction_phName="frontright",
    t_spatial=[0.0, -1.0, 0.0]
)

Kext2 = externalTangentFollowerTL(
    prob;
    F=Frearright,
    traction_phName="rearright",
    t_spatial=[0.0, 1.0, 0.0]
)

sparse(Int64[], Int64[], Float64[], 20577, 20577)

In [586]:
f_int = internalForceTL(prob, P)

nodal VectorField
[-0.00023174007875215063; -0.00029395442131610546; … ; 0.000982940809141149; -0.0005483780704482455;;]

In [587]:
K = Kint + Kext1 + Kext2

sparse([1, 2, 3, 46, 47, 48, 73, 74, 75, 76  …  20511, 20521, 20522, 20523, 20569, 20570, 20571, 20575, 20576, 20577], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577, 20577], [0.7274743931676745, 0.022308003086846287, -0.12258809254420222, -0.04100756913019919, -0.021779778668838474, -9.48751253957685e-5, -0.2172638340294011, 0.06056129974537203, 0.01069768464187514, -0.0418227088196642  …  -2.561977993482607, -0.23459281669054008, -2.3896843131915406, -20.992332474366066, -0.05021990469177563, 0.041614503779367526, 6.819130650226614, 0.0005479816612616005, 0.06440459756337212, 58.65257550475568], 20577, 20577)

In [588]:
Δu = solveField(
    K,
    f_ext - f_int;
    support=[suppX]
)

fixed = constrainedDoFs(prob, [suppX])
free  = freeDoFs(prob, [suppX])
#norm(Δu.a[:, 1])
r = f_ext - f_int
println("‖r_free‖  = ", norm(r.a[free, 1]))
println("‖r_fixed‖ = ", norm(r.a[fixed, 1]))
println("‖r‖ = ", norm(r.a[:,1]))
println("‖Δu‖ = ", norm(Δu.a[:,1]))


‖r_free‖  = 0.45201614341152047
‖r_fixed‖ = 0.06041340607551497
‖r‖ = 0.4560354959191985
‖Δu‖ = 7.25296014107114


In [589]:
u += Δu

nodal VectorField
[0.0; 0.0; … ; -0.8328200071782811; 0.04946527284141952;;]

In [590]:
showDoFResults(u)
openPostProcessor()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XRequest.18: BadValue 0x0


In [591]:
F = I + u ∘ ∇

elementwise TensorField
[[0.997119665226722; 0.08506730768370899; … ; -0.11940819030344385; 0.9852039931810204;;], [0.9971016055928794; 0.11515095094919392; … ; -0.38407743658330906; 0.9188106698486231;;], [0.996981630641156; 0.12604361887131588; … ; -0.6120182632320335; 0.7898080814290023;;], [0.9978641227201505; 0.11116109311987565; … ; -0.7747298895633435; 0.6356806041891523;;], [0.9986503645508092; 0.08816310705383024; … ; -0.8767609785291919; 0.48529808982445977;;], [0.9992460017852121; 0.0641019525344424; … ; -0.934946282247842; 0.3591342104495219;;], [0.9985863791598447; 0.05342360362987275; … ; -0.969275313582278; 0.2507947463296938;;], [1.0295300948038302; 0.026769386657338412; … ; -0.9748739605606572; 0.16062528552428668;;], [1.0326242302476718; -0.054453379882899; … ; -0.9473480507818115; 0.1516907785100191;;], [0.989415563372548; 0.08616277888236556; … ; -0.12191303934229625; 0.9881340204383453;;]  …  [1.018633282391356; 0.1018837581108869; … ; -0.9573627807102045; 0.134371

In [592]:
E = 0.5 * (F' * F - I)
S = λ * trace(E) * I + 2μ * E
P = F * S
#P = λ * trace(F - I) * I + 2μ * (F - I)

elementwise TensorField
[[0.1083720357088914; 0.6636094528374453; … ; -0.021557701679038758; -0.03390889948637692;;], [0.08842170445089763; 0.4801430341691605; … ; -0.053568811857673995; 0.04879175117962852;;], [0.11927350049875571; 0.5345038575027076; … ; -0.10872690926002537; 0.09912456144282125;;], [0.13487297845010277; 0.4780923756362966; … ; -0.16340825894128835; 0.11261291373762577;;], [0.1352905091992586; 0.3801988681320141; … ; -0.13348763879584596; 0.06155774234589096;;], [0.0799119072961984; 0.2670836485385061; … ; -0.09389277695397757; 0.029296381024832726;;], [0.03767923708830428; 0.22489375389084373; … ; -0.10502235990163662; 0.025596014241148775;;], [0.8214598485823571; 0.11554336709109414; … ; -0.01612832765604238; 0.016746696248203567;;], [1.0492287887884195; -0.5570785342215482; … ; 0.8576978481152505; -0.03446413301606195;;], [-0.13738629485872442; 0.6508263873192902; … ; -0.01627041739153865; -0.024355019521056852;;]  …  [0.9120626616409347; 0.8505687006092266; … ; 0

In [582]:
err = check_fint_Kint_consistency(prob, u; C=C1, bc=[suppX])

println("Relative tangent error = ", err)


‖df_fd‖  = 46.72850751970214
‖Kδu‖    = 36.86395648999341
rel err = 0.5390296048130927
Relative tangent error = 0.5390296048130927


In [583]:
#gmsh.finalize()